<a href="https://colab.research.google.com/github/mrkcrawford/hello-world/blob/master/Copy_of_IRL%26ERL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Step 1: Import libraries
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from scipy.stats import zscore

# Step 2: Download stock data
ticker = "TSLA"
data = yf.download(ticker, period="3y", interval="1d")

# Fix for MultiIndex columns
if isinstance(data.columns, pd.MultiIndex):
    data = data.xs(ticker, axis=1, level=1)

# Step 3: Calculate liquidity zones with improved logic
def calculate_liquidity_zones(df, window=20, threshold=1.5):
    df = df.copy()
    bin_size = df['Close'].std() * 0.005
    df['price_bin'] = (df['Close'] / bin_size).round() * bin_size

    volume_profile = df.groupby('price_bin')['Volume'].sum()
    volume_profile = volume_profile.sort_values(ascending=False)

    # Use z-score to find significant volume clusters
    z_scores = zscore(volume_profile)
    significant_zones = volume_profile[abs(z_scores) > threshold]

    return significant_zones.sort_index()

liquidity_zones = calculate_liquidity_zones(data)

# Step 4: Identify IRL and ERL with new approach
latest_close = data['Close'].iloc[-1]

# Find consolidation period (recent price range)
lookback_period = 20  # days
recent_data = data.iloc[-lookback_period:]
consolidation_high = recent_data['High'].max()
consolidation_low = recent_data['Low'].min()

# IRL - Internal Range Liquidity (within consolidation)
irl_high = consolidation_high
irl_low = consolidation_low

# ERL - External Range Liquidity (beyond consolidation)
# Find highest high and lowest low beyond consolidation range
full_range_high = data['High'].max()
full_range_low = data['Low'].min()

# Filter liquidity zones for ERL
erl_high_zones = liquidity_zones[liquidity_zones.index > irl_high]
erl_low_zones = liquidity_zones[liquidity_zones.index < irl_low]

# Select ERL levels - highest volume zone beyond consolidation
erl_high = erl_high_zones.idxmax() if len(erl_high_zones) > 0 else full_range_high
erl_low = erl_low_zones.idxmax() if len(erl_low_zones) > 0 else full_range_low

# Print identified levels
print("\nIdentified Levels:")
print(f"Current Price: {latest_close:.2f}")
print(f"Consolidation Range: {irl_low:.2f} - {irl_high:.2f}")
print(f"IRL High: {irl_high:.2f}")
print(f"IRL Low: {irl_low:.2f}")
print(f"ERL High: {erl_high:.2f}")
print(f"ERL Low: {erl_low:.2f}")

# Step 5: Create candlestick chart
fig = go.Figure()
fig.add_trace(go.Candlestick(
    x=data.index,
    open=data['Open'],
    high=data['High'],
    low=data['Low'],
    close=data['Close'],
    name='Price'
))

# Highlight consolidation range (IRL zone)
fig.add_shape(
    type="rect",
    x0=data.index[-lookback_period],
    x1=data.index[-1],
    y0=irl_low,
    y1=irl_high,
    fillcolor="LightSkyBlue",
    opacity=0.3,
    layer="below",
    line_width=0,
    name="IRL Zone"
)

# Mark IRL - Internal High and Low
fig.add_trace(go.Scatter(
    x=[data.index[0], data.index[-1]],
    y=[irl_high, irl_high],
    mode='lines',
    name='IRL High',
    line=dict(color='green', dash='dot', width=2)
))

fig.add_trace(go.Scatter(
    x=[data.index[0], data.index[-1]],
    y=[irl_low, irl_low],
    mode='lines',
    name='IRL Low',
    line=dict(color='red', dash='dot', width=2)
))

# Mark ERL - External Range Liquidity
fig.add_trace(go.Scatter(
    x=[data.index[0], data.index[-1]],
    y=[erl_high, erl_high],
    mode='lines',
    name='ERL High (Buy Stops)',
    line=dict(color='white', dash='dash', width=2)
))

fig.add_trace(go.Scatter(
    x=[data.index[0], data.index[-1]],
    y=[erl_low, erl_low],
    mode='lines',
    name='ERL Low (Sell Stops)',
    line=dict(color='yellow', dash='dash', width=2)
))

# Step 6: Final Layout
fig.update_layout(
    title=f'{ticker} - IRL and ERL Levels (Current: {latest_close:.2f})',
    xaxis_title='Date',
    yaxis_title='Price ($)',
    legend_title='Key Levels:',
    template='plotly_dark',
    margin=dict(l=100),
    xaxis_rangeslider_visible=False,
    height=600
)

fig.show()

/tmp/ipython-input-3-358372376.py:10: FutureWarning:

YF.download() has changed argument auto_adjust default to True

[*********************100%***********************]  1 of 1 completed


Identified Levels:
Current Price: 317.66
Consolidation Range: 273.21 - 357.54
IRL High: 357.54
IRL Low: 273.21
ERL High: 488.54
ERL Low: 194.62
